In [15]:
## CODE adapted from https://www.kaggle.com/shubhamagarwal92/stock-prediction

## We will be using LSTM in Keras for this problem
## Also used word embeddings  
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
## For RNNs, trying without stemming and removing stopwords
# from collections import Counter
# from nltk.stem.porter import PorterStemmer
# from nltk.corpus import stopwords
from string import punctuation



# Keras tokenizer and sequence padder 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense,LSTM,Flatten
from keras.layers.embeddings import Embedding

In [16]:
# Hyperparameters

MAX_TOP_WORDS = 10000 # 5000, 20000
MAX_WORDS = 400
EMBEDDING_DIM = 128 # 10, 100, 256
UNITS = 128

# variational dropout, to ensure that LSTM converges:
DROPOUT_W = 0.2
DROPOUT_U = 0.2

LSTM_ACTIVATION = 'tanh' # also sigmoid, but see: https://www.quora.com/In-an-LSTM-unit-what-is-the-reason-behind-the-use-of-a-tanh-activation
DENSE_ACTIVATION = 'sigmoid' # also None (would be linear)

LOSS = 'binary_crossentropy' # also 'categorical_hinge' https://isaacchanghau.github.io/post/loss_functions/

OPTIMIZER = 'adam' # also adadelta, sgd https://keras.io/optimizers/
NB_EPOCH = 5 # 1, 2, 5, 10, 20

BATCH_SIZE = 32





In [17]:
# satisfy compiler
trainX = 0
testX = 0

def join_reviews(df):
# Function to join multiple columns in each row to form corpus for each day
	joinedReviews = df.apply(lambda x: ''.join(str(x.values)), axis=1)
	return joinedReviews

def pre_process(text):
# Basic form of pre-processing. Mainly removing 'b' character 
# representing bytes till 467 row.   
# Also remove punctuations and convert to lower case
	process_text=text.replace('\n','').replace('"b','').replace("'b",'')
	for punc in list(punctuation):
		process_text=process_text.replace(punc,'').lower()
	# process_text = process_text.strip(" ")
	# Remove extra spaces
	process_text=re.sub(' +',' ',process_text)
	return process_text
## Fit tokenizer
def generate_sequence(text,MAX_SEQUENCE_LENGTH):
	sequence= tokenizer.texts_to_sequences(text)
	sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)
	return sequence

In [18]:
# Import data
df = pd.read_csv('Data1.csv', encoding = "ISO-8859-1")
# print df.head()
# print df.shape
# print df.iloc[0,3]
names = df.columns.values 
### Dividing the data into test and train by dates (as specified)
trainStartDate = '2008-08-08' 
trainEndDate = '2014-12-31'
testStartDate = '2015-01-02'
testEndDate = '2016-07-01'
train = df[(df[names[0]]>=trainStartDate) & (df[names[0]]<=trainEndDate)]
test = df[(df[names[0]]>=testStartDate) & (df[names[0]]<=testEndDate)]


In [19]:
## Converting train into train_x and train_y 
trainX = train.loc[:,names[2:len(names)-1]]
## Subset by indices
# trainX = train.loc[:,[2,3]]
## Subset by indices
trainY = train.iloc[:,1]
## Subset by name
# trainY = train['Label']
# Similarly for test data. testX contains only textual information
testX = test.loc[:,names[2:len(names)-1]]
testY = test.iloc[:,1]
# print trainY.value_counts()
# 1    873
# 0    738
# delete all the other variables
del(train,test,trainStartDate,trainEndDate,testStartDate,testEndDate)
## Joined the reviews together for each day
## Should devise a mechansim to weight the reviews
joinedTrainX = join_reviews(trainX)
joinedTestX = join_reviews(testX)
joinedTrainX = joinedTrainX.apply(lambda x: pre_process(x))
joinedTestX = joinedTestX.apply(lambda x: pre_process(x))

In [20]:

# Define max top words in vocabulary
tokenizer = Tokenizer(nb_words=MAX_TOP_WORDS)
tokenizer.fit_on_texts(joinedTrainX)
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))

In [21]:
## Maximum sequence length

sequencesTrainX = generate_sequence(joinedTrainX,MAX_WORDS)
sequencesTestX = generate_sequence(joinedTestX,MAX_WORDS)
# result = map(len, sequencesTrainX)
# print("Mean %.2f words (%f)" % (np.mean(result), np.std(result)))
## Building the model
## First we will use word embedding and then LSTM layer over it 



In [22]:
# Seed for reproducability
seed = 1
np.random.seed(seed)
## Dimension of embedding layer

model = Sequential()
model.add(Embedding(MAX_TOP_WORDS, EMBEDDING_DIM, input_length=MAX_WORDS,dropout=0.2))
# model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))
# model.add(MaxPooling1D(pool_length=2))
model.add(LSTM(UNITS,dropout_W=DROPOUT_W,dropout_U=DROPOUT_U, activation = LSTM_ACTIVATION))
model.add(Dense(1, activation=DENSE_ACTIVATION))
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=['accuracy'])
# print(model.summary())
# model.fit(sequencesTrainX, trainY, batch_size=batch_size, nb_epoch=1,verbose=1)
# scores = model.evaluate(X_test, y_test, verbose=1)
## Fit the model
model.fit(sequencesTrainX, trainY, batch_size=BATCH_SIZE, nb_epoch=NB_EPOCH,
          validation_data=(sequencesTestX, testY),verbose=1)
# Ideally should have separate validation data          
score, acc = model.evaluate(sequencesTestX, testY,
                            batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)

# Ideally should use grid search 
# Hyperparameters
# Vocabulary = 5000
# Max sequence length = 500
# Embedding dimension = 128
# Embedding dropout = 0.2
# Lstm Neurons = 128
# Epoch =5

# loss: 0.6653 - acc: 0.6102 - val_loss: 0.6924 - val_acc: 0.5132
# Test score: 0.692432944106
# Test accuracy: 0.513227514804

# Vocabulary = 10000
# loss: 0.4408 - acc: 0.8535 - val_loss: 0.7931 - val_acc: 0.5344
# Test score: 0.793134343056
# Test accuracy: 0.534391534549

# Vocabulary = 20000

# loss: 0.4714 - acc: 0.8597 - val_loss: 0.7335 - val_acc: 0.5106
# Test score: 0.733532233528
# Test accuracy: 0.510582010267

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  import sys
/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, activation="tanh", dropout=0.2, recurrent_dropout=0.2)`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1611 samples, validate on 378 samples
Epoch 1/5
1611/1611 [==============================] - 27s 17ms/step - loss: 0.6906 - acc: 0.5400 - val_loss: 0.6942 - val_acc: 0.5079
Epoch 2/5
1611/1611 [==============================] - 25s 15ms/step - loss: 0.6285 - acc: 0.6754 - val_loss: 0.7258 - val_acc: 0.4974
Epoch 3/5
1611/1611 [==============================] - 24s 15ms/step - loss: 0.3109 - acc: 0.9032 - val_loss: 0.9961 - val_acc: 0.5238
Epoch 4/5
1611/1611 [==============================] - 26s 16ms/step - loss: 0.0792 - acc: 0.9752 - val_loss: 1.1982 - val_acc: 0.5423
Epoch 5/5
378/378 [==============================] - 1s 3ms/step
Test score: 1.5100097542717343
Test accuracy: 0.5582010575703212
